# Prices 

In [169]:
# import data 
import pandas as pd 

In [170]:
pd.set_option('display.max_columns', None)

In [171]:
# read data 

# oil and coffee prices 
oil_p=pd.read_csv('USEIA\Europe_Brent_Spot_Price_FOB.csv', header=4)
coffee_p=pd.read_excel("FNC\Precios-área-y-producción-de-café.xlsx", sheet_name='2. Precio Interno Mensual',skiprows=5)[['Mes','Precio interno']]

# cpi index
cpi=pd.read_csv("OECD\CPI_colombia.csv")

# Exchange rate 
exrate=pd.read_csv("OECD\Exchangerate_COL_US.csv")

### Oil prices 

In [172]:
oil_p.rename(columns={'Year':'year'}, inplace=True)
oil_p.head(5)

,year,Europe Brent Spot Price FOB Dollars per Barrel
0,2023,82.49
1,2022,100.93
2,2021,70.86
3,2020,41.96
4,2019,64.30


In [173]:
# turn to pesos 
# keep relevant columns of exchange rate 
exrate=exrate[['TIME_PERIOD','OBS_VALUE']]

# rename columns 
exrate.rename(columns={'TIME_PERIOD':'year','OBS_VALUE':'COP/USD'}, inplace=True)
oil_p=oil_p.merge(exrate, how='inner',on='year', validate='1:1')

# tranform price of oil to pesos 
oil_p['poil_COP']=oil_p['Europe Brent Spot Price FOB Dollars per Barrel']*oil_p['COP/USD']

# keep relevant columns 
oil_p=oil_p[['year','poil_COP']]

In [174]:
oil_p

,year,poil_COP
0,2022,429577.677275
1,2021,265317.141674
2,2020,154969.853953
3,2019,211008.307074
4,2018,210859.921220
5,2017,159763.986492
6,2016,133331.328200
7,2015,143450.016817
8,2014,198049.361850
9,2013,202887.241200


### Coffee
* Internal price per 125kg

In [175]:
# extract year
coffee_p['year']=coffee_p['Mes'].dt.year

# get annunal mean price 
coffee_p=coffee_p[['year','Precio interno']].groupby('year').mean()['Precio interno'].reset_index()

# rename columns 
coffee_p.rename(columns={'Precio interno':'pcoffee_COP'}, inplace=True)
# only only years we are interested in 
coffee_p=coffee_p[coffee_p['year']>1987]

### Turn to 2006 pesos

In [176]:
# keep relevant columns 
cpi=cpi[['Unit of measure','TIME_PERIOD','OBS_VALUE']]

# only information on CPI index 
cpi=cpi[cpi['Unit of measure']=='Index'].reset_index().drop(['index','Unit of measure'],axis=1)

# rename columns 
cpi.rename(columns={'TIME_PERIOD':'year','OBS_VALUE':'CPI'}, inplace=True)

In [177]:
# extract 2006 CPI
cpi_2006=cpi[cpi['year']==2006]['CPI'].values

# create cpi tranformer column
cpi['frac']=cpi_2006/cpi['CPI']

In [178]:
# merge coffee and oil prices 
prices=pd.merge(coffee_p,oil_p,how='inner',on='year',validate='1:1')

# add cpi
prices=pd.merge(prices,cpi,how='inner',on='year',validate='1:1')

In [179]:
# turn prices to 2006 pesos
prices['pcoffee2006']=prices['pcoffee_COP']*prices['frac']
prices['poil2006']=prices['poil_COP']*prices['frac']

In [180]:
# check if reults make sense 
prices.head(5)

,year,pcoffee_COP,poil_COP,CPI,frac,pcoffee2006,poil2006
0,1988,49391.666667,4460.674395,4.917197,14.370663,709790.986715,64102.847584
1,1989,62372.666667,6974.190339,6.189156,11.417289,712126.733642,79626.343583
2,1990,76973.000000,11933.697600,7.993419,8.840195,680456.303960,105496.209859
3,1991,93598.750000,12568.136100,10.422450,6.779920,634592.062209,85210.960679
4,1992,90978.000000,13145.054294,13.237610,5.338077,485647.559162,70169.310523


In [181]:
# keep relevant columns 
prices_final=prices[['year','pcoffee2006','poil2006']]

# export to csv
prices_final.to_csv('poil_pcoffee_2006COP.csv')